In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline

# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm       

from itertools import combinations 

In [89]:
# setActions = ['a1']
# setResponse = [] #....
# setEffect = ['e1', 'e2'] #.... 

## This function reads the artificially genereted tables from the designed csv
def read_table(exp, row, freq):
    r1 = pd.Series(row)[3:7].tolist();
#     print(r1)
    r2 =pd.Series(row)[7:11].tolist();
#     print(r2)
    r3 = pd.Series(row)[11:15].tolist();
#     print(r3)
    frame = { 'r1': r1, 'r2': r2, 'r3':r3 } 
    tempTable = pd.DataFrame(frame).T
    # for each cells times a 'freq'
    tempTable = tempTable*freq
    return(tempTable)


## This function reads the artificially genereted tables from the designed csv
def create_one_response_table(response2effectTable, response):
    oneResponseTable = response2effectTable
    currentR = response

    oneResponseTable.loc['total'] = pd.Series(oneResponseTable.sum())
    oneResponseTable.loc['nonR'] = oneResponseTable.loc['total'] - oneResponseTable.loc[currentR]

    oneResponseTableWithTotal = oneResponseTable
    
    for other_response in myResponses:
        if other_response != currentR:
            oneResponseTable = oneResponseTable.drop(other_response)
    
    oneResponseTable = oneResponseTable.drop('total')
    return(oneResponseTable, oneResponseTableWithTotal)
#     print(oneResponseTable)

def checking_non_neg_assumptions(oneResponseTable, response2effectTableWithTotal):
    # Checking assumptions
    boolAssumptionTrue = True

    # Assumption 1: For each expected cell, the number > 0
    for aj in response2effectTableWithTotal.columns:
        if response2effectTableWithTotal.at['total', aj] == 0:
            boolAssumptionTrue = False


            
    
    # TODO 
#     if !boolAssumptionTrue:
#         print("NA")

    return(boolAssumptionTrue)


def checking_80perc_expected_assumptions(expected):
    # Assumption 2: At least 80% of expected cells >= 5 // test on the oneResponseTable
    # TO DISCUSS!!! VERY DIFFICULT, checking the full table or checking the one response table. 
    return((expected > 0).sum(1).sum() >= (0.8 * 8))

def chi2_cont_row(df):
    stat, p, dof, expected = chi2_contingency(df, correction=True)
#     print('dof=%d' % dof)
#     print(expected)

    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
#     print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

#     if abs(stat) >= critical:
#         print('Dependent (reject H0)')
#     else:
#         print('Independent (fail to reject H0)')


    # interpret p-value
    alpha = 1.0 - prob
#     print('significance=%.3f, p=%.3f' % (alpha, p))
#     if p <= alpha:
#         print('Dependent (reject H0)')
#     else:
#         print('Independent (fail to reject H0)')
    return stat, p, dof, expected 

def test_corrcoef():
    a = np.array([
        [1, 2, 3, 4],
        [1, 3, 1, 4],
        [8, 3, 8, 5],
        [2, 3, 2, 1]])

    r1, p1 = corrcoef(a)
    r2, p2 = corrcoef_loop(a)

    assert np.allclose(r1, r2)
    assert np.allclose(p1, p2)
    
def calculate_number_of_significant_edges(oneResponseTable, currentR):
    countEdgesDraw = 0
    countZeroEdgesDraw = 0
    countSigLessEdgesDraw = 0
    countSigMoreEdgesDraw = 0
    countdiffedges = 0
    sharededges = 0
    adjresidual = 2.57 
    dfgfilter = 0.8
    shortfull = table.iloc[0:3, 0:4]
    maxtable = max(shortfull.max())
    threshold = maxtable-(maxtable*dfgfilter)
    
    stat, p, dof, expected = chi2_cont_row(oneResponseTable)
    
    rtable = sm.stats.Table(oneResponseTable)     
#         print(rtable.resid_pearson)
#         print(rtable.standardized_resids)

    subtable = pd.DataFrame(rtable.standardized_resids)
#     print(subtable)
    
    expectedT = pd.DataFrame(expected, columns = oneResponseTable.columns, index = oneResponseTable.index)
#     print(newTable)
#     print(expectedT)

    for aj in oneResponseTable.columns:
        if (subtable.at[currentR,aj] < -adjresidual) or (subtable.at[currentR,aj] > adjresidual):
            countEdgesDraw +=1
            if(subtable.at[currentR,aj] < -adjresidual):
                countSigLessEdgesDraw += 1
            if (subtable.at[currentR,aj] > adjresidual) :
                countSigMoreEdgesDraw += 1
            if(oneResponseTable.at[currentR, aj] == 0):
                countZeroEdgesDraw +=1
#             if(oneResponseTable.at[currentR, aj] > threshold) and ((subtable.at[currentR,aj] <= adjresidual) or (subtable.at[currentR,aj] >= -adjresidual)):
#                 countdiffedges += 1
#             if(oneResponseTable.at[currentR, aj] <= threshold) and ((subtable.at[currentR,aj] > adjresidual) or (subtable.at[currentR,aj] < -adjresidual)):
#                 countdiffedges += 1
            if(oneResponseTable.at[currentR, aj] > threshold):
                sharededges += 1
    return(countEdgesDraw, countZeroEdgesDraw, countSigLessEdgesDraw, countSigMoreEdgesDraw, countdiffedges, sharededges)

In [124]:
header_list = ["nEff", "nResp", "step", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11", "c12"]

filenumber = 1
n = 10
for filenumber in range(filenumber, n):
    file = 'Archive/pmf_tables_for_evaluation_'+str(filenumber)+'.csv'
    expFullTable = pd.read_csv(file, ',',  names=header_list, encoding='latin1')
    
    #print(expFullTable.head())

    columnRes = 'Responses'

    expFullTable.count()
    
    # create 'table' from CSV
    rowNumber = 1
    freq = 100
    test = read_table(expFullTable, expFullTable.iloc[rowNumber], freq)
    # print(test)
    
    myResponses = {"r1", "r2", "r3"}
    create_one_response_table(test, 'r1')
    
    #calculate_number_of_significant_edges(test, 'r1')
    countRow = 0 

    for i, row in expFullTable.iterrows():
        if(countRow % 1000 == 0):
            print(countRow)
            countRow+=1
    
        orgtable = read_table(expFullTable, row, freq)
        table = orgtable.copy()
    #     print("This is table")
    #     print(table)
    #     print("----")
    
        totNumEdgesOfOurs = 0
        totNumZeroEdgesOurs = 0
        totNumSigMoreEdgesOurs = 0
        totNumDiffEdges = 0
        totNumSharedEdges = 0
    
        bool_Assumption_all_responses = True;
    
        pvalues = list()
        numEdges = list()
        numZeroEdges = list()
        numSigMoreEdges = list()
        numDiffEdges = list()
        numSharedEdges = list()
    
        # For each response 
        for response in myResponses:
            oneResponseTable, oneResponseTableWithTotal = create_one_response_table(table, response)
    #         print("This is one response:")
    #         print(oneResponseTable)
        
        
            # Test assumptions
            bool_Assumption_Single_Response = checking_non_neg_assumptions(oneResponseTable, oneResponseTableWithTotal)
            if(bool_Assumption_Single_Response):
                stat, p, dof, expected = chi2_cont_row(oneResponseTable)
            
                if (checking_80perc_expected_assumptions(expected)):
                    pvalues.append(p)
                    num, numZero, numSigLess, numSigMore, numDiff, numShared = calculate_number_of_significant_edges(oneResponseTable, response)
                    numEdges.append(num)
                    numZeroEdges.append(numZero)
                    numSigMoreEdges.append(numSigMore)
                    numDiffEdges.append(numDiff)
                    numSharedEdges.append(numShared)
                
                else:
                    bool_Assumption_all_responses = False;
            
            else:
                ## Chi-test's assumptions do not hold. We do not compute anything
                bool_Assumption_all_responses = False;
            
      
        expFullTable.at[i, 'ourNumEdges'] = str('NA')
        expFullTable.at[i, 'ourNumZeroEdges'] = str('NA')
        expFullTable.at[i, 'ourNumPosEdges'] = str('NA')
        expFullTable.at[i, 'ourNumSigMoreEdges'] = str('NA')
        expFullTable.at[i, 'DiffEdges'] = str('NA')
        expFullTable.at[i, 'SharedEdges'] = str('NA')
        if(bool_Assumption_all_responses):
        
            p_adjusted_list = multipletests(pvalues, alpha=.05, method='bonferroni')
            for index, p_adjusted in enumerate(p_adjusted_list[0]):
    #             print(p_adjusted)
                if(p_adjusted):
                    totNumEdgesOfOurs += numEdges[index]
                    totNumZeroEdgesOurs += numZeroEdges[index]
                    totNumSigMoreEdgesOurs += numSigMoreEdges[index]
                    totNumDiffEdges += numDiffEdges[index]
                    totNumSharedEdges += numSharedEdges[index]
        
            expFullTable.at[i, 'ourNumEdges'] = str(totNumEdgesOfOurs)
            expFullTable.at[i, 'ourNumZeroEdges'] = str(totNumZeroEdgesOurs)
            expFullTable.at[i, 'ourNumPosEdges'] = str(totNumEdgesOfOurs - totNumZeroEdgesOurs)
            expFullTable.at[i, 'ourNumSigMoreEdges'] = str(totNumSigMoreEdgesOurs)
            expFullTable.at[i, 'DiffEdges'] = str(totNumDiffEdges)
            expFullTable.at[i, 'SharedEdges'] = str(totNumSharedEdges)
    #     else:
    #         expFullTable.at[i, 'ourNumEdges'] = str('NA')
    #         expFullTable.at[i, 'ourNumZeroEdges'] = str('NA')
    #         expFullTable.at[i, 'ourNumPosEdges'] = str('NA')
    #         expFullTable.at[i, 'ourNumSigMoreEdges'] = str('NA')
    
        dfgfilter = 0.8
        shortfull = table.iloc[0:3, 0:4]
        maxtable = max(shortfull.max())
        threshold = maxtable-(maxtable*dfgfilter)
        dfgNumEdges80 = shortfull[shortfull > threshold].count().sum();
        dfgNumEdges100 = (orgtable > 0).sum(1).sum();
        #     print(dfgNumEdges.tolist())
  
        expFullTable.at[i, 'dfgNumEdges80'] = dfgNumEdges80
        expFullTable.at[i, 'dfgNumEdges100'] = dfgNumEdges100
    
    # standard deviation of each column (effect) and the average for all four columns
    expFullTable['sdE1'] = expFullTable[['c1','c5','c9']].std(axis=1)
    expFullTable['sdE2'] = expFullTable[['c2','c6','c10']].std(axis=1)
    expFullTable['sdE3'] = expFullTable[['c3','c7','c11']].std(axis=1)
    expFullTable['sdE4'] = expFullTable[['c4','c8','c12']].std(axis=1)
    expFullTable['sda'] = expFullTable[['sdE1','sdE2','sdE3','sdE4']].mean(axis=1)
    # round sda to 4 digits
    expFullTable = expFullTable.round({'sda': 4})

    #print(expFullTable.loc[1])
    file2 = "Archive/pmf_tables_for_evaluation_res_80per_"+str(filenumber)+".csv"
    expFullTable.to_csv(file2, ",", encoding="latin1")

0


In [2]:
#### for clean data ####
import os, glob
import pandas as pd

# path = 'Archive/'

# all_files = glob.glob(os.path.join(path, 'pmf_tables_for_evaluation_res_80per_*.csv'))

# df_merged = (pd.read_csv(f, sep=',') for f in all_files)
# df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
# df_merged = pd.concat(df_from_each_file, ignore_index=True)
# df_merged.to_csv('Archive/Merged/clean_80per.csv')

# TestEvalTableClean = pd.read_csv('Archive/Merged/clean_80per.csv', ',', encoding='latin1')

#### for noisy data ####
TestEvalTableNoise = pd.read_csv('Archiven/pmf_tables_for_evaluation_res_1.csv', ';', encoding = 'latin1')

In [3]:
##### choose data clean or noisy ####
TestEvalTable = TestEvalTableNoise
#print(TestEvalTable.describe())

In [4]:
TestEvalTable['OurEdgesNZ'] = TestEvalTable['ourNumEdges'] - TestEvalTable['ourNumZeroEdges']
TestEvalTable['OurEdgesNZWZ'] = TestEvalTable['OurEdgesNZ'].fillna(0)
TestEvalTable['DiffEdges80'] = (TestEvalTable['OurEdgesNZWZ'] - TestEvalTable['SharedEdges']) + (TestEvalTable['dfgNumEdges80'] - TestEvalTable['SharedEdges'])
TestEvalTable['DiffEdges100'] = (TestEvalTable['OurEdgesNZWZ'] - TestEvalTable['SharedEdges']) + (TestEvalTable['dfgNumEdges100'] - TestEvalTable['SharedEdges'])
TestEvalTable.loc[TestEvalTable.OurEdgesNZWZ == 0, "SharedEdges"] = 0
TestEvalTable.loc[TestEvalTable.OurEdgesNZWZ == 0, "DiffEdges80"] = TestEvalTable['dfgNumEdges80']
TestEvalTable.loc[TestEvalTable.OurEdgesNZWZ == 0, "DiffEdges100"] = TestEvalTable['dfgNumEdges100']
#print(TestEvalTable.describe())

In [5]:
#round up sda
TestEvalTable = TestEvalTable.round({'sda': 4})
# tables for (bubble) charts + tables (like table 5)
Bdfg80 = pd.crosstab(TestEvalTable['sda'], TestEvalTable['dfgNumEdges80'], margins=True, margins_name="Total")
BUBDFG80 = Bdfg80.drop('Total', axis = 1)
BUBDFG80 = BUBDFG80.stack()
print(BUBDFG80.head(n=20))
#print(Bdfg80.head(n=20))
BUBDFG80.to_csv('Archiven/Merged/BUBDFG80_noise_r.csv')
#Bdfg80.to_csv('Archive/Merged/Bdfg80_noise_r.csv')

Bdfg100 = pd.crosstab(TestEvalTable['sda'], TestEvalTable['dfgNumEdges100'], margins=True, margins_name="Total")
BUBDFG100 = Bdfg100.drop('Total', axis = 1)
BUBDFG100 = BUBDFG100.stack()
print(BUBDFG100.head(n=20))
#print(Bdfg100.head(n=20))
BUBDFG100.to_csv('Archiven/Merged/BUBDFG100_noise_r.csv')
#Bdfg100.to_csv('Archive/Merged/Bdfg100_noise_r.csv')

Bo = pd.crosstab(TestEvalTable['sda'], TestEvalTable['OurEdgesNZWZ'], margins=True, margins_name="Total")
BUBour = Bo.drop('Total', axis = 1)
BUBour = BUBour.stack()
print(BUBour.head(n=20))
#print(Bo.head(n=20))
BUBour.to_csv('Archiven/Merged/BUBour_noise_r.csv')
#Bo.to_csv('Archive/Merged/Bo_noise_r.csv')

sda     dfgNumEdges80
0.0000  3.0               4
        4.0               0
        5.0               0
        6.0              24
        7.0               0
        8.0               0
        9.0              24
        10.0              0
        11.0              0
        12.0              4
0.0491  3.0               6
        4.0               0
        5.0               0
        6.0              30
        7.0               9
        8.0               9
        9.0               6
        10.0              3
        11.0              3
        12.0              0
dtype: int64


C:\Users\Koorn004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


sda     dfgNumEdges100
0.0000  6.0                1
        7.0                0
        8.0                0
        9.0               18
        10.0               0
        11.0               0
        12.0              37
0.0491  6.0                0
        7.0                0
        8.0                0
        9.0                0
        10.0              18
        11.0              18
        12.0              30
0.0520  6.0                0
        7.0                0
        8.0                0
        9.0               12
        10.0              24
        11.0              24
dtype: int64


C:\Users\Koorn004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


sda     OurEdgesNZWZ
0.0000  0.0             56
        1.0              0
        2.0              0
        3.0              0
        4.0              0
        5.0              0
        6.0              0
        7.0              0
        8.0              0
        9.0              0
        10.0             0
        11.0             0
        12.0             0
0.0491  0.0              2
        1.0             20
        2.0             36
        3.0              6
        4.0              2
        5.0              0
        6.0              0
dtype: int64


C:\Users\Koorn004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [175]:
Bs = pd.crosstab(TestEvalTable['sda'], TestEvalTable['SharedEdges'], margins=True, margins_name="Total")
BUBshare = Bs.drop('Total', axis = 1)
BUBshare = BUBshare.stack()
# print(Bs.head(n=20))
# print(BUBshare.head(n=20))
Bs.to_csv('Archiven/Merged/Bshared_noise.csv')

Bd100 = pd.crosstab(TestEvalTable['sda'], TestEvalTable['DiffEdges100'], margins=True, margins_name="Total")
BUBdiff100 = Bd100.drop('Total', axis = 1)
BUBdiff100 = BUBdiff100.stack()
# print(Bd100.head(n=20))
# print(BUBdiff100.head(n=20))
Bd100.to_csv('Archiven/Merged/Bdifferent100_noise.csv')

Bd80 = pd.crosstab(TestEvalTable['sda'], TestEvalTable['DiffEdges80'], margins=True, margins_name="Total")
BUBdiff80 = Bd80.drop('Total', axis = 1)
BUBdiff80 = BUBdiff80.stack()
# print(Bd80.head(n=20))
# print(BUBdiff80.head(n=20))
Bd80.to_csv('Archiven/Merged/Bdifferent80_noise.csv')